In [1]:
# Dependencies and Setup
import json
import os
import pandas as pd
from pandas import DataFrame
import urllib.request
import requests
from sqlalchemy import create_engine

In [2]:
# Create reference to CSV file
csv_path2 = "Resources/Womply Revenue - State - Daily.csv"
womply_state_df = pd.read_csv(csv_path2, low_memory=False)
womply_state_df.head()

,year,month,day,statefips,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70
0,2020,1,10,1,-0.0154,0.00161,0.0457,-0.0544,0.00902
1,2020,1,10,2,0.0199,0.03600,-0.0334,-0.0294,0.03410
2,2020,1,10,4,-0.0119,-0.00727,-0.0145,-0.0315,0.00664
3,2020,1,10,5,-0.0201,-0.02100,-0.0369,-0.0115,-0.03130
4,2020,1,10,6,0.0024,-0.00617,0.0216,0.0228,-0.00764


In [4]:
#Pull Data from  to June
covid_revenue = womply_state_df.loc[womply_state_df["month"].isin([2, 3, 4, 5, 6, 7, 8])] 
covid_revenue.head()

,year,month,day,statefips,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70
1122,2020,2,1,1,-0.0211,-0.02430,-0.00398,-0.0258,-0.00287
1123,2020,2,1,2,-0.0889,-0.03990,-0.07910,-0.4130,-0.03180
1124,2020,2,1,4,-0.0333,-0.01110,0.05630,-0.0772,-0.04070
1125,2020,2,1,5,-0.0488,0.00923,-0.08170,-0.0260,0.00344
1126,2020,2,1,6,-0.0342,-0.01510,-0.05200,-0.0755,-0.02620


In [ ]:
#Pull data from states with highest cases: Texas, illinois, California, Wisconsin and Florida
#revenue_byState = covid_revenue.loc[covid_revenue["statefips"].isin([48, 17, 6, 55, 12])] 
# revenue_texas = covid_revenue.loc[covid_revenue["statefips"]==48] 
# revenue_texas.head()

In [7]:
#Replace  Statefips by states names
fips_csv = "Resources/state_fips_master.csv"
fips_df = pd.read_csv(fips_csv, low_memory=False)
fips_df.head()

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name
0,Alabama,AL,Alabama AL,1,40,3,6,1,South,East South Central
1,Alaska,AK,Alaska AK,2,40,4,9,2,West,Pacific
2,Arizona,AZ,Arizona AZ,4,40,4,8,4,West,Mountain
3,Arkansas,AR,Arkansas AR,5,40,3,7,5,South,West South Central
4,California,CA,California CA,6,40,4,9,6,West,Pacific


In [8]:
#Drop unecessary columns on fips dataframe
fips_df.drop({'state_abbr', 'long_name', 'sumlev', 'region', 'division', 'state', 'region_name', 'division_name'}, axis=1, inplace=True)
fips_df.head()

,state_name,fips
0,Alabama,1
1,Alaska,2
2,Arizona,4
3,Arkansas,5
4,California,6


In [9]:
#Rename fips to statefips
fips_df.rename(columns = {'fips':'statefips'}, inplace = True) 
fips_df.head()

,state_name,statefips
0,Alabama,1
1,Alaska,2
2,Arizona,4
3,Arkansas,5
4,California,6


In [11]:
#join both DataFrame to get state names on the revenue dataframe
revenue_by_state = pd.merge(covid_revenue,fips_df, on=["statefips"])
revenue_by_state.head()

,year,month,day,statefips,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70,state_name
0,2020,2,1,1,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287,Alabama
1,2020,2,2,1,-0.02320,-0.01600,0.00171,-0.0222,-0.00414,Alabama
2,2020,2,3,1,0.00782,0.00641,-0.01660,0.0536,0.00691,Alabama
3,2020,2,4,1,0.00456,0.00157,-0.04530,0.0687,0.00734,Alabama
4,2020,2,5,1,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190,Alabama


In [12]:
#Query data types: Note that ss60 has some objects
# revenue_byState.dtypes
revenue_by_state.dtypes

year              int64
month             int64
day               int64
statefips         int64
revenue_all     float64
revenue_ss40    float64
revenue_ss60    float64
revenue_ss65    float64
revenue_ss70    float64
state_name       object
dtype: object

In [ ]:
# covid_SB_rev.head()

In [19]:
#concatenate month, date and year to create a date

revenue_by_state['date'] = revenue_by_state['month'].astype(str) + '-' + revenue_by_state['day'].astype(str) + '-' + revenue_by_state['year'].astype(str) 
revenue_by_state.head()

,year,month,day,statefips,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70,state_name,date
0,2020,2,1,1,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287,Alabama,2-1-2020
1,2020,2,2,1,-0.02320,-0.01600,0.00171,-0.0222,-0.00414,Alabama,2-2-2020
2,2020,2,3,1,0.00782,0.00641,-0.01660,0.0536,0.00691,Alabama,2-3-2020
3,2020,2,4,1,0.00456,0.00157,-0.04530,0.0687,0.00734,Alabama,2-4-2020
4,2020,2,5,1,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190,Alabama,2-5-2020


In [26]:
#convert date column into datetime format
revenue_by_state['date'] = pd.to_datetime(revenue_by_state["date"].astype(str))
revenue_by_state.head()

,statefips,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70,state_name,date
0,1,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287,Alabama,2020-02-01
1,1,-0.02320,-0.01600,0.00171,-0.0222,-0.00414,Alabama,2020-02-02
2,1,0.00782,0.00641,-0.01660,0.0536,0.00691,Alabama,2020-02-03
3,1,0.00456,0.00157,-0.04530,0.0687,0.00734,Alabama,2020-02-04
4,1,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190,Alabama,2020-02-05


In [30]:
#Rearrange columns
revenue_by_state = revenue_by_state[['date', 'statefips', 'state_name','revenue_all', 'revenue_ss40', 'revenue_ss60', 'revenue_ss65', 'revenue_ss70']]
revenue_by_state.head()

,date,statefips,state_name,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70
0,2020-02-01,1,Alabama,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287
1,2020-02-02,1,Alabama,-0.02320,-0.01600,0.00171,-0.0222,-0.00414
2,2020-02-03,1,Alabama,0.00782,0.00641,-0.01660,0.0536,0.00691
3,2020-02-04,1,Alabama,0.00456,0.00157,-0.04530,0.0687,0.00734
4,2020-02-05,1,Alabama,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190


In [59]:
revenue_by_state.to_csv('state SB_data.csv', header=True, index=False) 
revenue_by_state.to_json(r'C:\Users\ideuk\Desktop\ETL project\state SB_data.json')

In [ ]:
#30 days average
# covid_SB_rev["Bi - revenue_all"] = covid_SB_rev.iloc[:,2].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss40"] = covid_SB_rev.iloc[:,3].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss60"] = covid_SB_rev.iloc[:,4].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss65"] = covid_SB_rev.iloc[:,5].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss70"] = covid_SB_rev.iloc[:,6].rolling(window=30).mean()
# covid_SB_rev

In [ ]:
# # #Save Dataframe into csv file
# revenue_texas.to_csv('SB_data.csv', header=True, index=False) 

In [ ]:
# #Save Json file
# SB_json_data = revenue_texas.to_json(r'C:\Users\ideuk\Desktop\ETL project\SB_data.json')

# COVID CASES

In [31]:
# Create reference to CSV file
csv_path = "Resources/COVID - State - Daily.csv"

Covid_bystate_df = pd.read_csv(csv_path, low_memory=False)
Covid_bystate_df.head()


,year,month,day,statefips,case_count,death_count,test_count,case_rate,death_rate,test_rate,new_positives_rate,new_case_rate,new_death_rate,new_test_rate,new_case_count,new_death_count,new_test_count
0,2020,1,21,1,0,0,.,0.0,0.0,.,.,.,.,.,.,.,.
1,2020,1,21,2,0,0,.,0.0,0.0,.,.,.,.,.,.,.,.
2,2020,1,21,4,0,0,.,0.0,0.0,.,.,.,.,.,.,.,.
3,2020,1,21,5,0,0,.,0.0,0.0,.,.,.,.,.,.,.,.
4,2020,1,21,6,0,0,.,0.0,0.0,.,.,.,.,.,.,.,.


In [32]:
#Drop COVID Columns - state
covid_cases_df = Covid_bystate_df.drop(columns=['death_count', 'new_death_count', 'test_count', 'case_rate', 'death_rate', 'test_rate', 'new_positives_rate', 'new_test_count', 'new_case_rate', 'new_death_rate', 'new_test_rate'])
covid_cases_df.head()

,year,month,day,statefips,case_count,new_case_count
0,2020,1,21,1,0,.
1,2020,1,21,2,0,.
2,2020,1,21,4,0,.
3,2020,1,21,5,0,.
4,2020,1,21,6,0,.


In [34]:
# covid_data = covid_cases_df.loc[covid_cases_df["month"].isin([2,3,4,5,6, 7, 8])] 
covid_data = covid_cases_df.loc[covid_cases_df["month"].isin([2,3,4,5,6, 7, 8])]
covid_data. head()

,year,month,day,statefips,case_count,new_case_count
561,2020,2,1,1,0,0
562,2020,2,1,2,0,0
563,2020,2,1,4,1,.143
564,2020,2,1,5,0,0
565,2020,2,1,6,3,.286


In [35]:
#join both DataFrame to get state names on the covid case dataframe
cases_by_state = pd.merge(covid_data,fips_df, on=["statefips"])
cases_by_state.head()

,year,month,day,statefips,case_count,new_case_count,state_name
0,2020,2,1,1,0,0,Alabama
1,2020,2,2,1,0,0,Alabama
2,2020,2,3,1,0,0,Alabama
3,2020,2,4,1,0,0,Alabama
4,2020,2,5,1,0,0,Alabama


In [37]:
#Query data types: Note that new case count has some objects
# covid_byState.dtypes
cases_by_state.dtypes

year               int64
month              int64
day                int64
statefips          int64
case_count         int64
new_case_count    object
state_name        object
dtype: object

In [43]:
#Replace "." by 0 - create a new dataframe
# covid_byState.replace(to_replace =".",
#                  value =0) 
cases_by_state.replace(to_replace =".",
                 value =0)
cases_by_state.head()

,year,month,day,statefips,case_count,new_case_count,state_name
0,2020,2,1,1,0,0,Alabama
1,2020,2,2,1,0,0,Alabama
2,2020,2,3,1,0,0,Alabama
3,2020,2,4,1,0,0,Alabama
4,2020,2,5,1,0,0,Alabama


In [44]:
cases_by_state.dtypes

year               int64
month              int64
day                int64
statefips          int64
case_count         int64
new_case_count    object
state_name        object
dtype: object

In [46]:
# covid_byState.new_case_count=covid_byState.new_case_count.astype('int64')
cases_by_state.new_case_count = cases_by_state.new_case_count.astype('float64')

In [47]:
# covid_byState.dtypes
cases_by_state.dtypes

year                int64
month               int64
day                 int64
statefips           int64
case_count          int64
new_case_count    float64
state_name         object
dtype: object

In [48]:
#concatenate month, date and year to create a date

cases_by_state['date'] = cases_by_state['month'].astype(str) + '-' + cases_by_state['day'].astype(str) + '-' + cases_by_state['year'].astype(str) 
cases_by_state.head()

,year,month,day,statefips,case_count,new_case_count,state_name,date
0,2020,2,1,1,0,0.0,Alabama,2-1-2020
1,2020,2,2,1,0,0.0,Alabama,2-2-2020
2,2020,2,3,1,0,0.0,Alabama,2-3-2020
3,2020,2,4,1,0,0.0,Alabama,2-4-2020
4,2020,2,5,1,0,0.0,Alabama,2-5-2020


In [49]:
#convert date column into datetime format
cases_by_state['date'] = pd.to_datetime(cases_by_state["date"].astype(str))
cases_by_state.head()

,year,month,day,statefips,case_count,new_case_count,state_name,date
0,2020,2,1,1,0,0.0,Alabama,2020-02-01
1,2020,2,2,1,0,0.0,Alabama,2020-02-02
2,2020,2,3,1,0,0.0,Alabama,2020-02-03
3,2020,2,4,1,0,0.0,Alabama,2020-02-04
4,2020,2,5,1,0,0.0,Alabama,2020-02-05


In [50]:
#Drop Year, month and date
cases_by_state.drop({'year', 'month', 'day'}, axis=1, inplace=True)


In [52]:
#Rearrange columns
cases_by_state[['date', 'statefips', 'state_name', 'case_count', 'new_case_count']]
cases_by_state.head()

,statefips,case_count,new_case_count,state_name,date
0,1,0,0.0,Alabama,2020-02-01
1,1,0,0.0,Alabama,2020-02-02
2,1,0,0.0,Alabama,2020-02-03
3,1,0,0.0,Alabama,2020-02-04
4,1,0,0.0,Alabama,2020-02-05


In [ ]:
#7 days average
# covid_SB_rev["Bi - revenue_all"] = covid_SB_rev.iloc[:,2].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss40"] = covid_SB_rev.iloc[:,3].rolling(window=30).mean()
# covid_SB_rev["Bi - revenue_ss60"] = covid_SB_rev.iloc[:,4].rolling(window=30).mean()
# covid_SB_rev

In [53]:
cases_by_state.to_csv('state covid_data.csv', header=True, index=False) 
cases_by_state.to_json(r'C:\Users\ideuk\Desktop\ETL project\state covid_data.json')


# CONNECT TO POSTGRESS - From previous project - to remove? 

In [54]:
rds_connection_string = f"postgres:postgres@localhost:5432/ETL_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [55]:
engine.table_names()

['covid_data', 'revenue_data']

In [57]:
#table_name = 'covid_data'
cases_by_state.to_sql(name='covid_data', con=engine, if_exists='replace', index=False)

In [60]:
#table_name = 'revenue_data'
revenue_by_state.to_sql(name='revenue_data', con=engine, if_exists='replace', index=False)

In [61]:
#Confirm data has been added by querying the customer_location table

pd.read_sql_query('select * from covid_data', con=engine).head()

,statefips,case_count,new_case_count,state_name,date
0,1,0,0.0,Alabama,2020-02-01
1,1,0,0.0,Alabama,2020-02-02
2,1,0,0.0,Alabama,2020-02-03
3,1,0,0.0,Alabama,2020-02-04
4,1,0,0.0,Alabama,2020-02-05


In [62]:
#Confirm data has been added by querying the customer_location table

pd.read_sql_query('select * from revenue_data', con=engine).head()

,date,statefips,state_name,revenue_all,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70
0,2020-02-01,1,Alabama,-0.02110,-0.02430,-0.00398,-0.0258,-0.00287
1,2020-02-02,1,Alabama,-0.02320,-0.01600,0.00171,-0.0222,-0.00414
2,2020-02-03,1,Alabama,0.00782,0.00641,-0.01660,0.0536,0.00691
3,2020-02-04,1,Alabama,0.00456,0.00157,-0.04530,0.0687,0.00734
4,2020-02-05,1,Alabama,-0.03180,-0.03050,-0.05070,-0.0164,-0.01190


In [64]:
#Join revenue and covid data with statefips id and save it as a dataframe

# covid_revenue = pd.read_sql_query('SELECT covid_data.year,covid_data.month,covid_data.day, covid_data.statefips,covid_data.new_case_count, revenue_data.revenue_all FROM covid_data INNER JOIN revenue_data ON covid_data.statefips=revenue_data.statefips;', con=engine)
# covid_revenue